In [35]:
import pandas as pd
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
from sklearn.model_selection import train_test_split
import numpy as np

In [36]:
train = train.drop(['from', 'color', 'style'], axis=1)
test = test.drop(['from', 'color', 'style'], axis=1)
train['features'] = len(train['features'])
test['features'] = len(test['features'])
train['len_description'] = len(train['description'])
test['len_description'] = len(test['description'])
train['wheel'] = 1 if 'Левый' else 0
test['wheel'] = 1 if 'Левый' else 0
train['engine_displacement'].replace('Electro', '10', inplace=True)
test['engine_displacement'].replace('Electro', '10', inplace=True)
import numpy as np
train['engine_displacement'] = train['engine_displacement'].astype(np.float64)
test['engine_displacement'] = test['engine_displacement'].astype(np.float64)
train['generation'].replace('Unknown', '-1', inplace=True)
test['generation'].replace('Unknown', '-1', inplace=True)
train['generation'] = train['generation'].astype(np.int64)
test['generation'] = test['generation'].astype(np.int64)

In [37]:
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

y = train['price']
X = train.drop('price', axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [38]:
from sklearn.model_selection import GridSearchCV

In [39]:
from catboost import CatBoostRegressor
estimator = CatBoostRegressor()

In [40]:
model = CatBoostRegressor(iterations=25_000, learning_rate=0.02, depth=5, loss_function='MAE', 
                          cat_features=['body', 'brand', 'fuel', 'transmission', 'drive_type', 'condition', 'description', 'brand'], 
                          custom_metric=['MAE'])

In [41]:
model.fit(X_train, y_train)

0:	learn: 498701.5927096	total: 35.9ms	remaining: 14m 56s
1:	learn: 491914.1899184	total: 74.4ms	remaining: 15m 29s
2:	learn: 485318.8686829	total: 116ms	remaining: 16m 7s
3:	learn: 479840.5470815	total: 152ms	remaining: 15m 50s
4:	learn: 473310.7327515	total: 184ms	remaining: 15m 21s
5:	learn: 467142.3438658	total: 218ms	remaining: 15m 6s
6:	learn: 461339.2543353	total: 256ms	remaining: 15m 15s
7:	learn: 455690.2391328	total: 292ms	remaining: 15m 11s
8:	learn: 449946.2037286	total: 326ms	remaining: 15m 6s
9:	learn: 443823.3566193	total: 358ms	remaining: 14m 54s
10:	learn: 439197.8406264	total: 392ms	remaining: 14m 50s
11:	learn: 433689.8384400	total: 424ms	remaining: 14m 42s
12:	learn: 428293.8740712	total: 461ms	remaining: 14m 45s
13:	learn: 423184.9763590	total: 491ms	remaining: 14m 35s
14:	learn: 417692.1302679	total: 520ms	remaining: 14m 26s
15:	learn: 412514.0538584	total: 551ms	remaining: 14m 20s
16:	learn: 407855.3897108	total: 588ms	remaining: 14m 24s
17:	learn: 403442.9207321

In [42]:
preds = model.predict(X_test)
mean_absolute_error(y_test, preds)

99093.304692108

In [43]:
sub = pd.DataFrame({'id': test.index})
preds = model.predict(test)
sub['price'] = preds
sub.to_csv('submission.csv', index=False)